In [34]:
!rm -rf xgb_model_*.json
!rm -rf xgb_model_*.pkl
!unzip -q xgb_models.zip
!unzip -q xgb_models_pkl.zip

unzip:  cannot find or open xgb_models.zip, xgb_models.zip.zip or xgb_models.zip.ZIP.


In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [2]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 92.7MB/s]


In [3]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [4]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 166.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 30.8 MB/s eta 0:00:00
   ━━━━

In [5]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [6]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [7]:


models_list_3=[
        # "FacebookAI/xlm-roberta-large",
      "FacebookAI/roberta-large",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
# "FacebookAI/xlm-roberta-large",
"FacebookAI/roberta-large",

# "bert",

]


In [8]:
kind="dev"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [9]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [10]:
all_csv

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,NaN,NaN,NaN,NaN,NaN,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,NaN,NaN,NaN,NaN,NaN,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,NaN,NaN,NaN,NaN,NaN,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,NaN,NaN,NaN,NaN,NaN,eng,its very dark and foggy
4,eng_dev_track_a_00005,NaN,NaN,NaN,NaN,NaN,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,NaN,NaN,NaN,NaN,NaN,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,NaN,NaN,NaN,NaN,NaN,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,NaN,NaN,NaN,NaN,NaN,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,NaN,NaN,NaN,NaN,NaN,eng,never been so close to a group asswooping in m...


In [11]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,NaN,NaN,NaN,NaN,NaN,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,NaN,NaN,NaN,NaN,NaN,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,NaN,NaN,NaN,NaN,NaN,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,NaN,NaN,NaN,NaN,NaN,eng,its very dark and foggy
4,eng_dev_track_a_00005,NaN,NaN,NaN,NaN,NaN,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,NaN,NaN,NaN,NaN,NaN,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,NaN,NaN,NaN,NaN,NaN,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,NaN,NaN,NaN,NaN,NaN,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,NaN,NaN,NaN,NaN,NaN,eng,never been so close to a group asswooping in m...


In [12]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [13]:
train_data=all_csv


In [14]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [15]:

train_data=train_data.fillna(0)

train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,0.0,0.0,0.0,0.0,0.0,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,0.0,0.0,0.0,0.0,0.0,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,0.0,0.0,0.0,0.0,0.0,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,0.0,0.0,0.0,0.0,0.0,eng,its very dark and foggy
4,eng_dev_track_a_00005,0.0,0.0,0.0,0.0,0.0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0.0,0.0,0.0,0.0,0.0,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,0.0,0.0,0.0,0.0,0.0,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,0.0,0.0,0.0,0.0,0.0,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,0.0,0.0,0.0,0.0,0.0,eng,never been so close to a group asswooping in m...


In [16]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,0,0,0,0,0,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,0,0,0,0,0,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,0,0,0,0,0,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,0,0,0,0,0,eng,its very dark and foggy
4,eng_dev_track_a_00005,0,0,0,0,0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,0,0,0,0,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,0,0,0,0,0,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,0,0,0,0,0,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,0,0,0,0,0,eng,never been so close to a group asswooping in m...


In [17]:
train_data.dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [18]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,0,eng_dev_track_a_00001,0,0,0,0,0,eng,older sister 23 at the time is a scumbag stacy
1,1,eng_dev_track_a_00002,0,0,0,0,0,eng,and i laughed like this garhahagar because my ...
2,2,eng_dev_track_a_00003,0,0,0,0,0,eng,it overflowed and brown shitty diarrhea water ...
3,3,eng_dev_track_a_00004,0,0,0,0,0,eng,its very dark and foggy
4,4,eng_dev_track_a_00005,0,0,0,0,0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...,...
111,111,eng_dev_track_a_00112,0,0,0,0,0,eng,my heart was beating fast from excitement
112,112,eng_dev_track_a_00113,0,0,0,0,0,eng,a fraying rope stretches down from the rafters
113,113,eng_dev_track_a_00114,0,0,0,0,0,eng,so i cried my eyes out and did the drawing
114,114,eng_dev_track_a_00115,0,0,0,0,0,eng,never been so close to a group asswooping in m...


In [19]:
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,0,0,0,0,0,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,0,0,0,0,0,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,0,0,0,0,0,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,0,0,0,0,0,eng,its very dark and foggy
4,eng_dev_track_a_00005,0,0,0,0,0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,0,0,0,0,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,0,0,0,0,0,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,0,0,0,0,0,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,0,0,0,0,0,eng,never been so close to a group asswooping in m...


In [20]:
label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [21]:
train_data['lang'].value_counts()

,count
lang,
eng,116


In [22]:
sum(train_data['lang'].isna())

0

In [23]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [24]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [25]:
train_data[train_data["lang"]=="deu"].dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [26]:
nan_rows["lang"].value_counts()

,count
lang,


In [27]:
nan_rows.dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [28]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

             id   fear    joy  sadness  surprise   lang  clean_message
lang                                                                  
eng  0    False  False  False    False     False  False          False
     1    False  False  False    False     False  False          False
     2    False  False  False    False     False  False          False
     3    False  False  False    False     False  False          False
     4    False  False  False    False     False  False          False
...         ...    ...    ...      ...       ...    ...            ...
     111  False  False  False    False     False  False          False
     112  False  False  False    False     False  False          False
     113  False  False  False    False     False  False          False
     114  False  False  False    False     False  False          False
     115  False  False  False    False     False  False          False

[116 rows x 7 columns]


<ipython-input-28-06fbda4bf1b6>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)


In [29]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [30]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [31]:
x_dev, y_dev = train_data['clean_message'].values.tolist(), train_data[label_columns].values.tolist()

In [32]:
y_dev_df = pd.DataFrame(y_dev, columns=label_columns)

In [58]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [59]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, encodings, labels):

        self.encodings = encodings

        self.labels = labels



    def __getitem__(self, idx):

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)

        return item


    def __len__(self):

        return len(self.labels)

In [60]:
!ls output/best-model-v1

ls: cannot access 'output/best-model-v1': No such file or directory


In [61]:
def build_dataset(x_train,y_train,x_valid,y_valid,MAX_LENGTH):

  train_encodings = tokenizer(x_train, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  valid_encodings = tokenizer(x_valid, truncation=True, padding="max_length", max_length=MAX_LENGTH)

  train_dataset = Dataset(train_encodings, y_train)

  valid_dataset = Dataset(valid_encodings, y_valid)

  return train_dataset,valid_dataset



def train_arg():

    training_args = TrainingArguments(

    output_dir="./output/best-model-v1",

    report_to=None,
    dataloader_pin_memory=False,

    num_train_epochs=3,

    gradient_accumulation_steps=2,

    per_device_train_batch_size=8,

    per_device_eval_batch_size=16,

    learning_rate=2e-5,

    weight_decay=0.01,

    eval_strategy=IntervalStrategy.STEPS,

    save_total_limit=1,

    logging_dir="./logs",

    logging_strategy=IntervalStrategy.STEPS,

    logging_steps=250,

    eval_steps=500,

    load_best_model_at_end=True,

    lr_scheduler_type="cosine_with_restarts",

    warmup_steps=500,

    save_strategy="steps",

    save_steps=500,

  )

    return training_args

In [62]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    """
    Compute metrics for multi-label classification.

    Args:
        eval_pred: Tuple of (predictions, labels).
            predictions: logits or probabilities of shape (batch_size, num_labels)
            labels: ground truth of shape (batch_size, num_labels)

    Returns:
        metrics: Dictionary containing accuracy, precision, recall, and F1-score.
    """
    # Unpack predictions and labels
    logits, labels = eval_pred

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=None
    )

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [63]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from scipy.special import expit  # For sigmoid activation if needed

def evaluate_model(model, valid_dataset, compute_metrics, batch_size=32, device="cuda",average=None):
    """
    Evaluate a multi-label classification model without using the Hugging Face Trainer.

    Args:
        model: The trained model (Hugging Face model).
        valid_dataset: The validation dataset (torch.utils.data.Dataset).
        compute_metrics: The function to compute evaluation metrics.
        batch_size: Batch size for evaluation.
        device: Device to perform evaluation (e.g., "cuda" or "cpu").

    Returns:
        metrics: Dictionary containing evaluation metrics.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # Create DataLoader for the validation dataset
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    all_predictions = []
    all_labels = []

    # Disable gradient computation for evaluation
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc="Evaluating"):
            # Get inputs and labels
            inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
            labels = batch["labels"].to(device)

            # Forward pass: Get logits
            logits = model(**inputs)

            # Apply sigmoid activation to logits if necessary (for multi-label classification)
            probabilities = expit(logits.cpu().numpy())  # Convert logits to probabilities
            predictions = (probabilities > 0.5).astype(int)  # Convert probabilities to binary predictions

            # Collect predictions and labels
            all_predictions.append(predictions)
            all_labels.append(labels.cpu().numpy())
    print()
    # Concatenate all predictions and labels
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    # Unpack predictions and labels
    logits, labels = all_predictions,all_labels

    # Convert logits to binary predictions
    # If your model outputs probabilities (e.g., after sigmoid), use threshold=0.5
    probabilities = expit(logits)
    predictions = (probabilities > 0.5).astype(int)

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute precision, recall, f1-score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average=average
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    },all_predictions

In [64]:
def evaluate_func(trainer):
    metrics = evaluate_model(
    model=model,
    valid_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    batch_size=32,
    device=device
)
    accuracy=metrics["accuracy"]
    precision=metrics["precision"]
    recall=metrics["recall"]
    f1=metrics["f1"]

    return precision, recall, f1, accuracy

In [65]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [66]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [67]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [68]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [69]:
USE_LORA=False

In [70]:
from transformers import AutoModel

In [71]:


torch.cuda.empty_cache()



print(x)
tokenizer = AutoTokenizer.from_pretrained(x)
embedding_model = AutoModel.from_pretrained(x, output_hidden_states=True)

google-bert/bert-base-uncased


In [72]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [73]:
# a=

In [74]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [75]:
if tokenizer.pad_token is None:
    print("adding tokenizer token")
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    tokenizer.pad_token=tokenizer.eos_token

    embedding_model.config.pad_token_id = tokenizer.pad_token_id

    # model.resize_token_embeddings(len(tokenizer))

    if embedding_model.config.pad_token_id is None:
        print("setting model pad token")
        embedding_model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.pad_token

'[PAD]'

In [76]:
device

'cuda'

In [77]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [78]:

# Custom Dataset for batching
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )


In [94]:

# Batch extraction function
def extract_embeddings(texts, tokenizer, model, batch_size=32, max_length=256):
    dataset = TextDataset(texts, tokenizer, max_length=max_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Pass input_ids through the embedding layer directly
            embedding_output = model.embeddings(input_ids)  # Access embedding layer

            pooled_embeddings = embedding_output.mean(dim=1)  # Mean pooling over sequence_length
            # Convert to numpy and store
            embedding_output_np = pooled_embeddings.cpu().numpy()
            embeddings.append(embedding_output_np)

    return np.vstack(embeddings)

In [95]:
# Extract embeddings for train and validation datasets
x_dev_embeddings = extract_embeddings(x_dev,tokenizer, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 4/4 [00:01<00:00,  2.14it/s]


In [96]:
# Flatten labels for XGBoost (multi-label classification)
y_dev_flat = np.array(y_dev)

In [97]:
import xgboost as xgb
import pickle

In [98]:
# Function to load XGBoost models
def load_xgb_models(label_columns):
    xgb_models = {}
    for label in label_columns:
        # model = xgb.XGBClassifier()
        # model.load_model(f"xgb_model_{label}.json")
        with open(f"xgb_model_{label}.pkl", 'rb') as f:
          model = pickle.load(f)
        xgb_models[label] = model
    return xgb_models

In [99]:
xgb_models = load_xgb_models(label_columns)

In [100]:

# Predict labels for the test data
test_predictions = {}
for label in label_columns:
    print(f"Predicting for label: {label}")
    test_predictions[label] = xgb_models[label].predict(x_dev_embeddings)

Predicting for label: anger
Predicting for label: fear
Predicting for label: joy
Predicting for label: sadness
Predicting for label: surprise


In [101]:
# Combine predictions into a DataFrame
predictions_df = pd.DataFrame(test_predictions)

In [102]:
train_data.reset_index(inplace=True)

In [103]:
import pickle
# Save the XGBoost models if needed
for label, model in xgb_models.items():
    with open(f"xgb_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [104]:
# Add the predictions to the original test DataFrame
output_df = pd.concat([train_data[['id','lang']], predictions_df], axis=1)

In [105]:
# Create directory if it doesn't exist
output_dir = task
os.makedirs(output_dir, exist_ok=True)

# Iterate through the unique languages in the 'lang' column
for lang in output_df['lang'].unique():
    # Filter rows for the current language
    lang_df = output_df[output_df['lang'] == lang]
    lang_df.drop(columns=['lang'],inplace=True)

    # Define the output file path for the current language
    lang_csv_file = os.path.join(output_dir, f"pred_{lang}.csv")

    # Sort the DataFrame by the 'id' column
    lang_df.sort_values(by='id', inplace=True)

    # Save the filtered DataFrame to a CSV file
    lang_df.to_csv(lang_csv_file, index=False)

    # Print a confirmation message
    print(f"Predictions for language '{lang}' saved to {lang_csv_file}")

Predictions for language 'eng' saved to track_a/pred_eng.csv


In [106]:
!zip -r {task}.zip {task}

  adding: track_a/ (stored 0%)
  adding: track_a/.ipynb_checkpoints/ (stored 0%)
  adding: track_a/pred_eng.csv (deflated 89%)
